### Use package = scrape_linkedin

In [1]:
import pandas as pd
import requests
from time import sleep
from selenium import webdriver
import chromedriver_binary
from scrape_linkedin import ProfileScraper
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

### Pre define parameter

In [20]:
#created a linkedin profile: yeomangoh@yahoo.com, password:08075323
#get li_at cookie (need login): https://www.smartwriter.ai/university/how-to-access-your-linkedin-cookie-and-what-exactly-does-a-cookie-do
li_at_value = 'AQEDATd4RK4C9QUEAAABe5xdggMAAAF7wGoGA1YAP8VFRmt1hVKulFeHCP-2ztYgJis24Yuq_Bxd7-tMlg8g3NG06cHybRecFh3tJjtGKT2l8s3fWVbctPC2z22jnR0oUFjhuL0-c0fF8FPzku2jlYTx'

#get names from organization page
companyUrl = "https://healthaxis.com/leadership-team"

companyName = "HealthAxis Group"

#number of col for final table
colLen = 8

timeSleep = 3

### UDF

In [74]:
def useProfileScraper(userName):
    try:
        with ProfileScraper(cookie = li_at_value) as scraper:
            profile = scraper.scrape(user ='%s'%userName)

        profileDict = profile.to_dict()

        #personal info
        colList = ['phone', 'email', 'company', 'school']
        resultDict = {}
        for col in colList:
            if col in profileDict['personal_info'].keys():
                resultDict[col] = profileDict['personal_info'][col]

        #working exp
        #skill
        resultDict['skills'] = ','.join([i['name'] for i in profileDict['skills']])

        #username
        resultDict['username'] = username
        
        #store result
        return resultDict
    
    except:
        pass

### Processing

In [69]:
#get usernamelist and name list 
options = webdriver.ChromeOptions()
options.add_argument('headless')  
driver = webdriver.Chrome(options = options)
driver.get(companyUrl)

className = "wpb_text_column wpb_content_element "
a_Tag = driver.find_elements_by_xpath("//div[@class = '%s']/div/p/span/a"%className)
FirstSpan_Tag = driver.find_elements_by_xpath("//div[@class = '%s']/div/p/span[1]"%className)
# SecondSpan_Tag = driver.find_elements_by_xpath("//div[@class = '%s']/div/p/span[@style = 'font-size: 18px;']"%className)
SecondSpan_Tag = driver.find_elements_by_xpath("//div[@class = '%s']/div/p/span[2]"%className)
hrefList = [i.get_attribute("href") for i in a_Tag]

#get nameList and jobList
nameList = [i.get_attribute("innerText") for i in a_Tag]
jobTitleList = [i.get_attribute("innerText") for i in SecondSpan_Tag]

In [73]:
dfResult.columns

Index(['name', 'phone', 'email', 'company', 'school', 'current title',
       'skills', 'username'],
      dtype='object')

In [78]:
resultList = []

for no, name in enumerate(nameList):
    #search and click
    url = 'https://www.linkedin.com'
    options = webdriver.ChromeOptions()
    options.add_argument('headless')  
    driver = webdriver.Chrome()
    driver.get(url)
    # driver.add_cookie({"name":"li_at", "value":li_at_value})

    email = "yeomangoh@yahoo.com"
    password = "08075323"
    #key in email and password
    emailBox = driver.find_element_by_xpath("//*[@id = 'session_key']")
    emailBox.send_keys(email)
    sleep(timeSleep)
    passwordBox = driver.find_element_by_xpath("//*[@id = 'session_password']")
    passwordBox.send_keys(password)

    #click sign in button
    sleep(timeSleep)
    signInButton = driver.find_element_by_xpath("//button[@type = 'submit']")
    signInButton.click()

    #key in text
    sleep(timeSleep)
    searchText = name + ' ' + companyName
    inputBox_xPath = "//input[@placeholder = 'Search']"
    wait = WebDriverWait(driver, 10)
    auto_element = wait.until(EC.visibility_of_element_located((By.XPATH, inputBox_xPath)))
    inputBox = driver.find_element_by_xpath(inputBox_xPath)
    inputBox.send_keys("%s"%searchText)
    inputBox.send_keys(Keys.RETURN)

    #click result link
    sleep(timeSleep)
    resultLink_xPath = "//a[@class = 'app-aware-link']"
    try:
        #here may fail if no results even fuzzy
        wait = WebDriverWait(driver, 10)
        auto_element = wait.until(EC.visibility_of_element_located((By.XPATH, resultLink_xPath)))
        resultLink = driver.find_element_by_xpath(resultLink_xPath)
        # resultLink.click()

        #get url from href to pass to scrape_linkedin func:
        currentUrl = resultLink.get_attribute("href")
        username = re.search(".*/(.*)\?", currentUrl).group(1)

    except:
        #if not even fuzzy match
        currentUrl = ''
        username = ''

    #if no exact match (return results/all) or not even fuzzy match (username = '')
    if re.search("results\/all", currentUrl) or username == '':
        tempDict = {}
        tempDict['username'] = username
        tempDict['name'] = name
        tempDict['jobTitle'] = jobTitleList[no]
        for col in ['phone', 'email', 'company', 'school', 'skills']:
            tempDict[col] = ''
        resultList.append(tempDict)
    else:
        tempDict = useProfileScraper(username)
        tempDict['name'] = name
        tempDict['jobTitle'] = jobTitleList[no]
        resultList.append(tempDict)

    driver.close()

In [79]:
dfResult = pd.DataFrame(resultList)
dfResult

,phone,email,company,school,skills,username,name,jobTitle
0,None,None,The HealthAxis Group,Columbia University - Columbia Business School,"Start-ups,Business Development,Management,Entr...",shilenpatel,Shilen Patel,Chairman & CEO
1,None,None,HealthAxis Group,State University of New York College at Potsdam,,jason-patchen-084062145,Jason Patchen,President
2,None,None,HealthAxis Group,Florida State University - College of Business,"Start-ups,Management,Business Development,New ...",jimclark01,Jim Clark,Chief Growth Officer
3,None,None,HealthAxis Group,West Virginia Wesleyan College,"Process Improvement,Business Analysis,Software...",zacharykilburn,Zachary Kilburn,Chief Technology Officer
4,,,,,,,Michele Mahoney,Chief Healthcare Operations Officer
5,,,,,,,Eric Strikowski,Chief Innovation Officer
6,,,,,,,Lisa Davis,Chief Data Officer
7,None,None,HealthAxis Group,Western New England College School of Law,"Medicare,Medicaid,Policy,Insurance,Health Insu...",michael-friel-268b6757,Michael Friel,Compliance Officer


In [81]:
filename = "Data/2021-09-01_LinkedInExample.csv"
dfResult.to_csv(filename, index = False)

In [ ]:
#possible side to find healthcare companies in ohio
#https://insurancecareers.org/ohio-companies

### Past Codes

In [24]:
# resultList = []
# failedList = []
# for no, username in enumerate(userNameList):
#     result = useProfileScraper(username)
#     if isinstance(result, dict):
#         resultList.append(result)
#     else:
#         failedList.append(nameList[no])
        
# #form df
# dfResult = pd.DataFrame(resultList)

# #get result where not match company - not the person
# notMatchList = list(dfResult[dfResult['company'].map(lambda x:re.search(companyName, x) == None)]
#                     ['name'].values)

# failedList2 = failedList + notMatchList

In [30]:
resultList2 = []

for name in failedList2:
    #search and click
    url = 'https://www.linkedin.com'
    options = webdriver.ChromeOptions()
    options.add_argument('headless')  
    driver = webdriver.Chrome()
    driver.get(url)
    # driver.add_cookie({"name":"li_at", "value":li_at_value})

    email = "yeomangoh@yahoo.com"
    password = "08075323"
    #key in email and password
    emailBox = driver.find_element_by_xpath("//*[@id = 'session_key']")
    emailBox.send_keys(email)
    sleep(timeSleep)
    passwordBox = driver.find_element_by_xpath("//*[@id = 'session_password']")
    passwordBox.send_keys(password)

    #click sign in button
    sleep(timeSleep)
    signInButton = driver.find_element_by_xpath("//button[@type = 'submit']")
    signInButton.click()

    #key in text
    sleep(timeSleep)
    searchText = name + ' ' + companyName
    inputBox_xPath = "//input[@placeholder = 'Search']"
    wait = WebDriverWait(driver, 10)
    auto_element = wait.until(EC.visibility_of_element_located((By.XPATH, inputBox_xPath)))
    inputBox = driver.find_element_by_xpath(inputBox_xPath)
    inputBox.send_keys("%s"%searchText)
    inputBox.send_keys(Keys.RETURN)

    #click result link
    sleep(timeSleep)
    resultLink_xPath = "//a[@class = 'app-aware-link']"
    try:
        #here may fail if no results even fuzzy
        wait = WebDriverWait(driver, 10)
        auto_element = wait.until(EC.visibility_of_element_located((By.XPATH, resultLink_xPath)))
        resultLink = driver.find_element_by_xpath(resultLink_xPath)
        # resultLink.click()

        #get url from href to pass to scrape_linkedin func:
        currentUrl = resultLink.get_attribute("href")
        username = re.search(".*/(.*)\?", currentUrl).group(1)

    except:
        #if not even fuzzy match
        currentUrl = ''
        username = ''

    #if no exact match (return results/all) or not even fuzzy match (username = '')
    if re.search("results\/all", currentUrl) or username == '':
        tempDict = {}
        tempDict['username'] = username
        tempDict['name'] = name
        for col in [i for i in dfResult.columns if i not in ['name', 'username']]:
            tempDict[col] = ''
        resultList2.append(tempDict)
    else:
        resultList2.append(useProfileScraper(username))

    driver.close()

TimeoutException: Message: 
